In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# DB connection
engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

In [4]:
# Helper Functions
def execute_query(query, params=None):
    with engine.connect() as conn:
        return pd.read_sql(query, conn, params=params)

def execute_update(query, params=None):
    with engine.begin() as conn:
        result = conn.execute(query, params)
        return result.rowcount

In [5]:
# Example 1: Batch Insert Single Record (CREATE Operations)
insert_query_1 = text("""
INSERT INTO earthquakes_enhanced (
    id, place, datetime_local, latitude, longitude, region, depth, magnitude, 
    tsunami, depth_category, alert_level, magnitude_hazard, hazard_score, intensity_score
)
VALUES (
    'test_001', 'Test Location - Central Java', '2025-06-03 10:00:00+07', 
    -7.5, 110.2, 'Java', 45.5, 5.2, 
    0, 'Shallow', 2, 'Strong', 12.5, 8.0
)
RETURNING *;
""")

new_record = execute_query(insert_query_1)
print("Inserted Record:")
display(new_record)

Inserted Record:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,test_001,Test Location - Central Java,2025-06-03 03:00:00+00:00,-7.5,110.2,Java,45.5,5.2,0,Shallow,2.0,Strong,12.5,8


In [6]:
# Example 2: Batch Insert (CREATE Operations)
batch_insert_query = text("""
INSERT INTO earthquakes_enhanced (
    id, place, datetime_local, latitude, longitude, region, depth, magnitude, 
    tsunami, depth_category, alert_level, magnitude_hazard, hazard_score, intensity_score
)
VALUES
    ('test_002', 'West Sumatra Coast', '2025-06-03 11:00:00+07', 
     0.5, 100.2, 'Sumatra', 15.2, 6.1, 
     1, 'Very Shallow', 3, 'Major', 18.5, 12.0),
    ('test_003', 'Banda Sea Region', '2025-06-03 12:00:00+07', 
     -6.8, 129.5, 'Eastern_Indonesia', 180.5, 4.8, 
     0, 'Intermediate', 1, 'Moderate', 9.2, 5.5),
    ('test_004', 'North Sulawesi', '2025-06-03 13:00:00+07', 
     1.2, 124.8, 'Sulawesi', 25.8, 5.5, 
     0, 'Very Shallow', 2, 'Strong', 15.3, 9.8)
RETURNING *;
""")

batch_records = execute_query(batch_insert_query)
print("Batch Inserted Records:")
display(batch_records)

Batch Inserted Records:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,test_002,West Sumatra Coast,2025-06-03 04:00:00+00:00,0.5,100.2,Sumatra,15.2,6.1,1,Very Shallow,3.0,Major,18.5,12
1,test_003,Banda Sea Region,2025-06-03 05:00:00+00:00,-6.8,129.5,Eastern_Indonesia,180.5,4.8,0,Intermediate,1.0,Moderate,9.2,6
2,test_004,North Sulawesi,2025-06-03 06:00:00+00:00,1.2,124.8,Sulawesi,25.8,5.5,0,Very Shallow,2.0,Strong,15.3,10


In [7]:
# Example 3: Conditional Insert (CREATE Operations)
conditional_insert_query = text("""
WITH high_risk_check AS (
    SELECT id FROM earthquakes_enhanced
    WHERE region = 'Java' 
    AND magnitude >= 6.0
    AND depth_category IN ('Very Shallow', 'Shallow')
    AND datetime_local >= NOW() - INTERVAL '24 hours'
)
INSERT INTO earthquakes_enhanced (
    id, place, datetime_local, latitude, longitude, region, depth, magnitude, 
    tsunami, depth_category, alert_level, magnitude_hazard, hazard_score, intensity_score
)
SELECT
    'test_005',
    'Central Java High Risk Zone',
    '2025-06-03 14:00:00+07',
    -7.8,
    110.5,
    'Java',
    22.5,
    6.2,
    1,
    'Very Shallow',
    4,
    'Major',
    22.8,
    15.5
WHERE NOT EXISTS (SELECT 1 FROM high_risk_check)
RETURNING *;
""")

conditional_insert = execute_query(conditional_insert_query)
print("Conditional Insert Result:")
display(conditional_insert)

Conditional Insert Result:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,test_005,Central Java High Risk Zone,2025-06-03 07:00:00+00:00,-7.8,110.5,Java,22.5,6.2,1,Very Shallow,4.0,Major,22.8,16


In [8]:
# Example 1: Select by Magnitude Range (READ Operations)
read_query_1 = """
SELECT *
FROM earthquakes_enhanced
WHERE magnitude BETWEEN 4.0 AND 5.0
ORDER BY datetime_local DESC
LIMIT 5;
"""

magnitude_range_results = execute_query(read_query_1)
print("Earthquakes with magnitude 4.0-5.0:")
display(magnitude_range_results)

Earthquakes with magnitude 4.0-5.0:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us6000qhlq,"120 km E of Bitung, Indonesia",2025-06-03 09:11:56.563000+00:00,1.3538,126.2067,Eastern_Indonesia,56.411,4.9,0,Shallow,0.0,Moderate,9.8,0
1,us6000qhhq,Banda Sea,2025-06-02 20:45:53.088000+00:00,-6.7432,129.4903,Eastern_Indonesia,166.353,4.6,0,Intermediate,0.0,Moderate,9.2,0
2,us6000qheu,"121 km NE of Lospalos, East Timor",2025-06-02 13:10:06.503000+00:00,-7.7410,127.7792,Eastern_Indonesia,176.730,4.1,0,Intermediate,0.0,Moderate,8.2,0
3,us6000qhci,"240 km SSE of Ambon, Indonesia",2025-06-02 04:14:29.462000+00:00,-5.6874,129.0606,Eastern_Indonesia,10.000,4.4,0,Very Shallow,0.0,Moderate,10.8,0
4,us6000qh9v,"20 km N of Pariaman, Indonesia",2025-06-01 16:13:11.080000+00:00,-0.4352,100.1471,Sumatra,82.130,4.6,0,Intermediate,0.0,Moderate,9.2,0


In [9]:
# Example 2: Geographic Query (READ Operations)
read_query_2 = """
SELECT *
FROM earthquakes_enhanced
WHERE longitude BETWEEN 100 AND 110
  AND latitude BETWEEN -5 AND 5
  AND depth_category = 'Shallow'
ORDER BY datetime_local DESC
LIMIT 5;
"""

geographic_results = execute_query(read_query_2)
print("\nShallow earthquakes in specified region:")
display(geographic_results)


Shallow earthquakes in specified region:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us7000q1qx,"80 km W of Bengkulu, Indonesia",2025-05-27 08:09:39.031000+00:00,-3.7624,101.5430,Sumatra,40.84,4.9,0,Shallow,0.0,Moderate,11.3,1
1,us7000pz3s,"63 km S of Bengkulu, Indonesia",2025-05-14 03:16:57.920000+00:00,-4.3720,102.3469,Sumatra,56.75,4.8,0,Shallow,0.0,Moderate,9.6,0
2,us7000pyul,"46 km WSW of Sungai Penuh, Indonesia",2025-05-12 15:40:06.363000+00:00,-2.1826,100.9906,Sumatra,64.33,4.6,0,Shallow,0.0,Moderate,9.2,0
3,us6000pzsd,"72 km NE of Tuapejat, Indonesia",2025-03-10 19:50:01.659000+00:00,-1.6069,100.0943,Sumatra,38.00,4.3,0,Shallow,0.0,Moderate,8.6,0
4,us6000pxts,"78 km SSW of Sungai Penuh, Indonesia",2025-03-09 18:29:21.184000+00:00,-2.7290,101.1611,Sumatra,49.74,4.5,0,Shallow,0.0,Moderate,9.0,0


In [10]:
# Example 3: Statistical Analysis (READ Operations)
read_query_3 = """
SELECT
    depth_category,
    COUNT(*) as event_count,
    ROUND(AVG(magnitude)::numeric, 2) as avg_magnitude,
    ROUND(MAX(magnitude)::numeric, 2) as max_magnitude,
    ROUND(MIN(depth)::numeric, 2) as min_depth,
    ROUND(MAX(depth)::numeric, 2) as max_depth,
    ROUND(AVG(hazard_score)::numeric, 2) as avg_hazard_score
FROM earthquakes_enhanced
WHERE datetime_local >= NOW() - INTERVAL '30 days'
GROUP BY depth_category
ORDER BY event_count DESC;
"""

stats_results = execute_query(read_query_3)
print("\nStatistical Analysis by Depth Category:")
display(stats_results)


Statistical Analysis by Depth Category:


,depth_category,event_count,avg_magnitude,max_magnitude,min_depth,max_depth,avg_hazard_score
0,Intermediate,23,4.41,4.9,82.13,214.77,8.82
1,Very Shallow,18,4.52,4.9,10.00,27.37,11.21
2,Shallow,15,4.59,4.9,31.09,66.87,9.37
3,Deep,5,4.40,4.8,355.03,546.06,8.80


In [11]:
# Example 1: Update Depth Categories (Update Operations)
update_query_1 = text("""
UPDATE earthquakes_enhanced
SET depth_category =
    CASE
        WHEN depth <= 30 THEN 'Very Shallow'
        WHEN depth <= 70 THEN 'Shallow'
        WHEN depth <= 300 THEN 'Intermediate'
        ELSE 'Deep'
    END
WHERE depth > 30 AND depth <= 70 AND depth_category != 'Shallow';
""")

rows_updated = execute_update(update_query_1)
print(f"Number of depth categories updated: {rows_updated}")

# Check updated records
select_query_1 = text("""
SELECT *
FROM earthquakes_enhanced
WHERE depth_category = 'Shallow'
ORDER BY depth
LIMIT 10;
""")

updated_categories = execute_query(select_query_1)
print("Updated records with Shallow category:")
display(updated_categories)

Number of depth categories updated: 0
Updated records with Shallow category:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us6000q61t,"64 km ESE of Masohi, Indonesia",2025-04-14 20:43:33.954000+00:00,-3.5425,129.4948,Eastern_Indonesia,30.493,4.5,0,Shallow,0.0,Moderate,9.0,0
1,us6000pk72,"45 km ESE of Waingapu, Indonesia",2025-01-03 17:58:10.466000+00:00,-9.8201,120.6467,Other,30.613,4.1,0,Shallow,0.0,Moderate,8.2,0
2,us7000pej7,"Pulau Pulau Tanimbar, Indonesia",2025-02-10 04:21:39.126000+00:00,-7.4576,130.4735,Eastern_Indonesia,31.074,4.2,0,Shallow,0.0,Moderate,8.4,0
3,us6000qh48,"21 km SSW of Palu, Indonesia",2025-05-31 18:41:49.243000+00:00,-1.0799,119.7834,Sulawesi,31.094,4.7,0,Shallow,0.0,Moderate,9.4,0
4,us6000q4x6,"209 km NNW of Tobelo, Indonesia",2025-04-09 15:21:22.331000+00:00,3.5137,127.3729,Other,31.630,5.0,0,Shallow,0.0,Moderate,10.0,0
5,us6000pvmq,"149 km W of Abepura, Indonesia",2025-02-18 06:44:06.958000+00:00,-2.4358,139.2945,Eastern_Indonesia,32.020,4.3,0,Shallow,0.0,Moderate,8.6,0
6,us6000pwms,"87 km WSW of Tambolaka, Indonesia",2025-03-03 15:20:57.449000+00:00,-9.8638,118.5771,Other,33.349,4.7,0,Shallow,0.0,Moderate,9.4,0
7,us6000pyst,"216 km NNW of Tobelo, Indonesia",2025-03-14 12:29:11.124000+00:00,3.5606,127.3263,Other,33.583,4.5,0,Shallow,0.0,Moderate,9.0,0
8,us6000pwft,"64 km WSW of Ternate, Indonesia",2025-03-02 12:26:37.915000+00:00,0.6365,126.8249,Eastern_Indonesia,34.779,4.9,0,Shallow,0.0,Moderate,9.8,0
9,us7000putk,"205 km NNW of Tobelo, Indonesia",2025-04-11 10:33:25.890000+00:00,3.3849,127.1707,Other,35.000,4.4,0,Shallow,0.0,Moderate,8.8,0


In [12]:
# Example 2: Update Place Names (Update Operations)
update_query_2 = text("""
UPDATE earthquakes_enhanced
SET place = REPLACE(place, 'Timor Leste', 'East Timor')
WHERE place LIKE '%Timor Leste%';
""")

rows_updated = execute_update(update_query_2)
print(f"\nNumber of place names updated: {rows_updated}")

# Check updated records
select_query_2 = text("""
SELECT *
FROM earthquakes_enhanced
WHERE place LIKE '%East Timor%'
LIMIT 10;
""")

updated_places = execute_query(select_query_2)
print("Updated place names:")
display(updated_places)


Number of place names updated: 0
Updated place names:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us7000p9t1,"294 km NE of Lospalos, East Timor",2025-01-29 08:06:15.096000+00:00,-7.0354,129.2127,Eastern_Indonesia,144.469,4.4,0,Intermediate,0.0,Moderate,8.8,0
1,us6000pmje,"270 km ENE of Lospalos, East Timor",2025-01-27 10:50:12.259000+00:00,-7.2631,129.1024,Eastern_Indonesia,125.047,4.9,0,Intermediate,0.0,Moderate,9.8,0
2,us6000pjd0,"179 km N of Metinaro, East Timor",2025-01-11 02:50:36.469000+00:00,-6.9038,125.7385,Eastern_Indonesia,523.601,4.1,0,Deep,0.0,Moderate,8.2,0
3,us6000pma3,"297 km ENE of Lospalos, East Timor",2025-01-25 10:06:57.016000+00:00,-7.0941,129.2823,Eastern_Indonesia,164.481,4.5,0,Intermediate,0.0,Moderate,9.0,0
4,us6000plwi,"16 km SSW of Suai, East Timor",2025-01-23 19:37:03.133000+00:00,-9.4545,125.2225,Eastern_Indonesia,10.000,4.1,0,Very Shallow,0.0,Moderate,10.2,0
5,us6000pl4s,"287 km ENE of Lospalos, East Timor",2025-01-19 21:21:43.537000+00:00,-7.3866,129.3398,Eastern_Indonesia,135.793,4.4,0,Intermediate,0.0,Moderate,8.8,0
6,us6000pm4u,"181 km NE of Lospalos, East Timor",2025-01-16 07:49:31.137000+00:00,-7.4245,128.2229,Eastern_Indonesia,125.830,4.2,0,Intermediate,0.0,Moderate,8.4,0
7,us6000pm4g,"37 km SSW of Suai, East Timor",2025-01-16 00:43:14.501000+00:00,-9.6173,125.1123,Eastern_Indonesia,15.772,4.1,0,Very Shallow,0.0,Moderate,10.2,0
8,us6000pi5h,"295 km ENE of Lospalos, East Timor",2025-01-06 12:29:34.624000+00:00,-7.2511,129.3531,Eastern_Indonesia,138.613,4.7,0,Intermediate,0.0,Moderate,9.4,0
9,us6000pvtp,"245 km ENE of Lospalos, East Timor",2025-02-27 14:35:44.146000+00:00,-7.3012,128.8586,Eastern_Indonesia,150.281,4.4,0,Intermediate,0.0,Moderate,8.8,0


In [13]:
# Example 3: Update Depths for Sumatra Region (Update Operations)
update_query_3 = text("""
UPDATE earthquakes_enhanced
SET depth = round(depth::numeric, 2)
WHERE region = 'Sumatra'
AND depth != round(depth::numeric, 2);
""")

rows_updated = execute_update(update_query_3)
print(f"Number of Sumatra records updated: {rows_updated}")

# Check updated records
select_query_3 = text("""
SELECT *
FROM earthquakes_enhanced
WHERE region = 'Sumatra'
ORDER BY depth
LIMIT 10;
""")

updated_depths = execute_query(select_query_3)
print("Updated depth precision for Sumatra region:")
display(updated_depths)

Number of Sumatra records updated: 0
Updated depth precision for Sumatra region:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us7000pzwb,"40 km WNW of Sinabang, Indonesia",2025-05-17 21:20:15.068000+00:00,2.5791,96.0257,Sumatra,10.00,4.9,0,Very Shallow,0.0,Moderate,11.8,0
1,us6000q6m1,"34 km W of Sinabang, Indonesia",2025-04-16 21:19:27.632000+00:00,2.4244,96.0764,Sumatra,10.00,4.2,0,Very Shallow,0.0,Moderate,10.4,0
2,us7000pu93,"18 km E of Padang, Indonesia",2025-04-08 10:23:34.845000+00:00,-0.9650,100.5216,Sumatra,10.00,4.3,0,Very Shallow,0.0,Moderate,10.6,0
3,us7000pabw,"78 km SW of Gunungsitoli, Indonesia",2025-01-22 03:21:38.976000+00:00,0.8550,97.0611,Sumatra,10.00,4.2,0,Very Shallow,0.0,Moderate,10.4,0
4,us7000pm8p,"41 km WSW of Blangpidie, Indonesia",2025-03-24 14:26:25.228000+00:00,3.5613,96.5119,Sumatra,10.00,4.5,0,Very Shallow,0.0,Moderate,11.0,0
5,us6000pzl8,"22 km WNW of Padangsidempuan, Indonesia",2025-03-09 07:31:29.615000+00:00,1.4822,99.0946,Sumatra,10.00,4.3,0,Very Shallow,0.0,Moderate,10.6,0
6,us7000pfiu,"169 km S of Pagar Alam, Indonesia",2025-02-22 23:44:52.037000+00:00,-5.5397,102.9862,Sumatra,10.00,4.7,0,Very Shallow,0.0,Moderate,11.4,0
7,us7000pvx7,"251 km WSW of Bengkulu, Indonesia",2025-04-14 19:58:43.988000+00:00,-4.4874,100.1104,Sumatra,10.00,4.2,0,Very Shallow,0.0,Moderate,10.4,0
8,us6000py63,"112 km SW of Teluk Dalam, Indonesia",2025-03-02 17:03:09.660000+00:00,-0.1809,97.1290,Sumatra,10.00,4.2,0,Very Shallow,0.0,Moderate,10.4,0
9,us7000p9u5,"143 km SSE of Tuapejat, Indonesia",2025-01-29 12:29:38.382000+00:00,-3.2148,100.1061,Sumatra,11.08,4.5,0,Very Shallow,0.0,Moderate,11.0,0


In [ ]:
# Example 1: Delete Duplicates (Delete Operations)

# Ambil data duplikat yang akan dihapus
select_duplicates_query = text("""
SELECT * FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY place, datetime_local, latitude, longitude, region,
                            depth, magnitude, tsunami, depth_category, alert_level,
                            magnitude_hazard, hazard_score, intensity_score
               ORDER BY id
           ) AS rn
    FROM earthquakes_enhanced
) sub
WHERE rn > 1
ORDER BY datetime_local DESC;
""")

deleted_duplicates = execute_query(select_duplicates_query)

# Hapus data duplikat
delete_query_1 = text("""
DELETE FROM earthquakes_enhanced
WHERE ctid IN (
    SELECT ctid FROM (
        SELECT ctid,
               ROW_NUMBER() OVER (
                   PARTITION BY place, datetime_local, latitude, longitude, region,
                                depth, magnitude, tsunami, depth_category, alert_level,
                                magnitude_hazard, hazard_score, intensity_score
                   ORDER BY id
               ) AS rn
        FROM earthquakes_enhanced
    ) sub
    WHERE rn > 1
);
""")

rows_deleted = execute_update(delete_query_1)

# Tampilkan hasil duplikat yang dihapus
print(f"Deleted duplicate records: {rows_deleted}")
display(deleted_duplicates)


Deleted duplicate records: 1200


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score,rn
0,us6000qhlq,"120 km E of Bitung, Indonesia",2025-06-03 09:11:56.563000+00:00,1.3538,126.2067,Eastern_Indonesia,56.411,4.9,0,Shallow,0.0,Moderate,9.8,0,2
1,us6000qhlq,"120 km E of Bitung, Indonesia",2025-06-03 09:11:56.563000+00:00,1.3538,126.2067,Eastern_Indonesia,56.411,4.9,0,Shallow,0.0,Moderate,9.8,0,3
2,us6000qhhq,Banda Sea,2025-06-02 20:45:53.088000+00:00,-6.7432,129.4903,Eastern_Indonesia,166.353,4.6,0,Intermediate,0.0,Moderate,9.2,0,2
3,us6000qhhq,Banda Sea,2025-06-02 20:45:53.088000+00:00,-6.7432,129.4903,Eastern_Indonesia,166.353,4.6,0,Intermediate,0.0,Moderate,9.2,0,3
4,us6000qheu,"121 km NE of Lospalos, Timor Leste",2025-06-02 13:10:06.503000+00:00,-7.7410,127.7792,Eastern_Indonesia,176.730,4.1,0,Intermediate,0.0,Moderate,8.2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,us6000pjrq,"66 km SSW of Labuha, Indonesia",2025-01-01 19:12:09.633000+00:00,-1.1688,127.2235,Eastern_Indonesia,10.000,4.4,0,Very Shallow,0.0,Moderate,10.8,0,3
1196,us6000pgx5,"51 km SSW of Labuha, Indonesia",2025-01-01 18:34:29.511000+00:00,-1.0739,127.3492,Eastern_Indonesia,10.000,4.5,0,Very Shallow,0.0,Moderate,11.0,0,2
1197,us6000pgx5,"51 km SSW of Labuha, Indonesia",2025-01-01 18:34:29.511000+00:00,-1.0739,127.3492,Eastern_Indonesia,10.000,4.5,0,Very Shallow,0.0,Moderate,11.0,0,3
1198,us6000pgru,"256 km SSE of Ambon, Indonesia",2025-01-01 01:07:10.492000+00:00,-5.9077,128.8869,Eastern_Indonesia,277.355,4.4,0,Intermediate,0.0,Moderate,8.8,0,3


In [17]:
# Example 2: Delete by Criteria (Delete Operations)
delete_query_2 = """
DELETE FROM earthquakes_enhanced
WHERE magnitude < 4.2
  AND hazard_score < 9
  AND datetime_local < NOW() - INTERVAL '7 days'
RETURNING *;
"""

deleted_old_records = execute_query(delete_query_2)
print("\nDeleted old, low-significance records:")
display(deleted_old_records)


Deleted old, low-significance records:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
0,us7000pad7,"203 km NW of Tobelo, Indonesia",2025-01-23 05:12:51.007000+00:00,3.2296,126.9458,Other,47.595,4.0,0,Shallow,0.0,Low,8.0,0
1,us6000pl98,"70 km SSW of Singaparna, Indonesia",2025-01-08 04:41:08.217000+00:00,-7.9024,107.7966,Java,72.443,4.1,0,Intermediate,0.0,Moderate,8.2,0
2,us7000pfiz,"202 km W of Gorontalo, Indonesia",2025-02-23 00:18:02.896000+00:00,0.6013,121.2448,Sulawesi,85.947,3.9,0,Intermediate,0.0,Low,7.8,0
3,us7000pf4p,"100 km SSE of Amahai, Indonesia",2025-02-21 00:40:14.374000+00:00,-4.2000,129.2082,Eastern_Indonesia,144.311,4.1,0,Intermediate,0.0,Moderate,8.2,0
4,us7000peqi,"220 km SSW of Palimbang, Philippines",2025-02-19 12:54:06.338000+00:00,4.4620,123.2228,Other,528.593,4.1,0,Deep,0.0,Moderate,8.2,0
5,us7000plnm,"Kepulauan Babar, Indonesia",2025-03-14 14:30:20.111000+00:00,-7.2988,129.4606,Eastern_Indonesia,148.201,4.1,0,Intermediate,0.0,Moderate,8.2,0
6,us6000pwit,"99 km SE of Taliwang, Indonesia",2025-03-03 00:02:05.772000+00:00,-9.4816,117.3686,Other,65.413,3.9,0,Shallow,0.0,Low,7.8,0
7,us7000pvnf,"296 km SE of Latung, Philippines",2025-04-28 20:36:22.537000+00:00,3.4386,122.5849,Other,553.521,4.1,0,Deep,0.0,Moderate,8.2,0
8,us7000pvne,"293 km SSE of Tabiauan, Philippines",2025-04-28 20:32:40.525000+00:00,3.4874,122.6520,Other,547.875,4.0,0,Deep,0.0,Low,8.0,0
9,us7000pwn8,"191 km NW of Tobelo, Indonesia",2025-04-18 03:34:10.284000+00:00,3.1176,126.9766,Other,52.982,4.1,0,Shallow,0.0,Moderate,8.2,0


In [18]:
# Example 3: Delete Invalid Coordinates (Delete Operations)
delete_query_3 = """
WITH invalid_records AS (
    DELETE FROM earthquakes_enhanced
    WHERE (latitude < -90 OR latitude > 90)
    OR (longitude < -180 OR longitude > 180)
    OR magnitude < 0 OR magnitude > 10
    OR depth < 0 OR depth > 800
    OR hazard_score < 0 OR hazard_score > 50
    OR intensity_score < 0 OR intensity_score > 30
    OR region IS NULL OR region = ''
    OR place IS NULL OR place = ''
    RETURNING *
)
SELECT * FROM invalid_records;
"""

deleted_invalid = execute_query(delete_query_3)
print("\nDeleted records with invalid coordinates:")
display(deleted_invalid)


Deleted records with invalid coordinates:


,id,place,datetime_local,latitude,longitude,region,depth,magnitude,tsunami,depth_category,alert_level,magnitude_hazard,hazard_score,intensity_score
